In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
import os, os.path 
import numpy
import pickle
from glob import glob
from typing import Any, Dict, List, Tuple, Union
import pandas as pd

"""Change to the data folder"""
new_path = "./new_train/new_train"
test_path = './new_val_in/new_val_in'
# number of sequences in each dataset
# train:205942  val:3200 test: 36272 
# sequences sampled at 10HZ rate

### Create a dataset class 

In [2]:
class ArgoverseDataset(Dataset):
    """Dataset class for Argoverse"""
    def __init__(self, data_path: str, transform=None):
        super(ArgoverseDataset, self).__init__()
        self.data_path = data_path
        self.transform = transform

        self.pkl_list = glob(os.path.join(self.data_path, '*'))
        self.pkl_list.sort()
        
    def __len__(self):
        return len(self.pkl_list)

    def __getitem__(self, idx):

        pkl_path = self.pkl_list[idx]
        with open(pkl_path, 'rb') as f:
            data = pickle.load(f)
            
        if self.transform:
            data = self.transform(data)

        return data


# intialize a dataset
val_dataset  = ArgoverseDataset(data_path=new_path)
test_dataset = ArgoverseDataset(data_path=test_path)

### Create a loader to enable batch processing

In [3]:
batch_sz = 6

def train_collate(batch):
    """ collate lists of samples into batches, create [ batch_sz x agent_sz x seq_len x feature] """
    inp = [numpy.dstack([scene['p_in'][scene['track_id'][:,0,0]==scene['agent_id'],:,:]]) for scene in batch]
    out = [numpy.dstack([scene['p_out'][scene['track_id'][:,0,0]==scene['agent_id'],:,:]]) for scene in batch]
    inp = torch.Tensor(inp)
    out = torch.Tensor(out)
    return [inp, out]

def test_collate(batch):
    """ collate lists of samples into batches, create [ batch_sz x agent_sz x seq_len x feature] """
    inp = [numpy.dstack([scene['p_in'][scene['track_id'][:,0,0]==scene['agent_id'],:,:]]) for scene in batch]
    inp = torch.Tensor(inp)
    idx = [numpy.dstack([scene['scene_idx']]) for scene in batch]
    return inp, idx
    
val_loader = DataLoader(val_dataset,batch_size=batch_sz, shuffle = True, collate_fn=train_collate, num_workers=0)

test_loader = DataLoader(test_dataset,batch_size=batch_sz, shuffle = True, collate_fn=test_collate, num_workers=0)

In [4]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

class EncoderRNN(nn.Module):
    """referenced from official Argoverse forecasting code: https://github.com/jagjeet-singh/argoverse-forecasting"""
    
    def __init__(self,
                 input_size = 2,
                 embedding_size = 8,
                 hidden_size = 16):
        
        super(EncoderRNN, self).__init__()
        
        self.hidden_size = hidden_size
        self.linear = nn.Linear(input_size, embedding_size)
        self.lstm = nn.LSTMCell(embedding_size, hidden_size)

    def forward(self, x, hidden):
        embedded = F.relu(self.linear(x))
        hidden = self.lstm(embedded, hidden)
        return hidden


class DecoderRNN(nn.Module):
    """Decoder Network."""
    """referenced from official Argoverse forecasting code: https://github.com/jagjeet-singh/argoverse-forecasting"""
    def __init__(self, embedding_size=8, hidden_size=16, output_size=2):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.linear1 = nn.Linear(output_size, embedding_size)
        self.lstm = nn.LSTMCell(embedding_size, hidden_size)
        self.linear2 = nn.Linear(hidden_size, output_size)

    def forward(self, x, hidden):
        embedded = F.relu(self.linear1(x))
        hidden = self.lstm(embedded, hidden)
        output = self.linear2(hidden[0])
        return output, hidden


In [5]:
from tqdm import tqdm_notebook as tqdm

def train(encoder, decoder, device, train_loader, encoder_optimizer, decoder_optimizer, epoch, log_interval=10000):    
    """referenced from official Argoverse forecasting code: https://github.com/jagjeet-singh/argoverse-forecasting"""
    
    iterator = tqdm(train_loader, total=int(len(train_loader)))
    counter = 0
    criterion = nn.MSELoss()
    
    for i_batch, sample_batch in enumerate(train_loader):
        
        inp, out = sample_batch
        # preprocessing more ????
        inp = inp[:,0,:,:]
        out = out[:,0,:,:]
        
        #inp - inp[0] for all in whaetver
        x_offset = []
        y_offset = []
        for i in range(inp.shape[0]):
            x_offset.append(inp[i][0][0].detach().clone())
            y_offset.append(inp[i][0][1].detach().clone())
    
        for j in range(inp.shape[0]):
            for i in range(inp.shape[1]):
                inp[j][i][0] = inp[j][i][0] - x_offset[j]
                inp[j][i][1] = inp[j][i][1] - y_offset[j]

        #outoput whatever
        for j in range(out.shape[0]):
            for i in range(out.shape[1]):
                out[j][i][0] = out[j][i][0] - x_offset[j]
                out[j][i][1] = out[j][i][1] - y_offset[j]
        
        _input, target = inp.to(device), out.to(device)
        
        encoder.train()
        decoder.train()
        
        encoder_optimizer.zero_grad()
        decoder_optimizer.zero_grad()
        
        
        #encoder 
        batch_size = _input.shape[0]
        input_length = _input.shape[1]
        output_length = target.shape[1]
        feature_len = _input.shape[2]
        input_shape = _input.shape[2]
        
        encoder_hidden = (torch.zeros(batch_size, encoder.module.hidden_size).to(device), 
                          torch.zeros(batch_size, encoder.module.hidden_size).to(device))
        
        loss = 0
        
        # Encode observed trajectory
        for ei in range(input_length):
            encoder_input = _input[:, ei, :]
            encoder_hidden = encoder(encoder_input, encoder_hidden)

        # Initialize decoder input with last coordinate in encoder
        decoder_input = encoder_input[:, :2]

        # Initialize decoder hidden state as encoder hidden state
        decoder_hidden = encoder_hidden

        decoder_outputs = torch.zeros(target.shape).to(device)

        # Decode hidden state in future trajectory
        for di in range(30):
            decoder_output, decoder_hidden = decoder(decoder_input,
                                                     decoder_hidden)
            decoder_outputs[:, di, :] = decoder_output

            # Update loss
            loss += torch.sqrt(criterion(decoder_output[:, :2], target[:, di, :2]))

            # Use own predictions as inputs at next step
            decoder_input = decoder_output

        # Get average loss for pred_len
        loss = loss / 30

        # Backpropagate
        loss.backward()
        encoder_optimizer.step()
        decoder_optimizer.step()
        
#         output = model(data)
#         loss = MSELoss(output, target)
        counter += 1
        iterator.set_postfix(loss=(loss.item()))

In [6]:
device = "cuda"
encoder = EncoderRNN(input_size=2)
decoder = DecoderRNN(output_size=2)

encoder = nn.DataParallel(encoder)
decoder = nn.DataParallel(decoder)

encoder.to(device)
decoder.to(device)

encoder_optimizer = torch.optim.Adam(encoder.parameters())
decoder_optimizer = torch.optim.Adam(decoder.parameters())

num_epoch = 20

for epoch in range(1, num_epoch + 1):
        train(encoder, decoder, device, val_loader, encoder_optimizer, decoder_optimizer, epoch)
#         predict(model, device, test_loader)

In [7]:
def infer_absolute(
        test_loader: torch.utils.data.DataLoader,
        encoder: EncoderRNN,
        decoder: DecoderRNN,
#         start_idx: int,
#         forecasted_save_dir: str,
#         model_utils: ModelUtils,
):
    """Infer function for non-map LSTM baselines and save the forecasted trajectories.
    
    referenced from official Argoverse forecasting code: https://github.com/jagjeet-singh/argoverse-forecasting
    
    Args:
        test_loader: DataLoader for the test set
        encoder: Encoder network instance
        decoder: Decoder network instance
        start_idx: start index for the current joblib batch
        forecasted_save_dir: Directory where forecasted trajectories are to be saved
        model_utils: ModelUtils instance

    """
    
    forecasted_trajectories = {}

    for i, (_input, idx) in enumerate(test_loader):
        
        _input = _input[:,0,:,:]
        
        #inp - inp[0] for all in whaetver
        x_offset = []
        y_offset = []
        for i in range(_input.shape[0]):
            x_offset.append(_input[i][0][0].detach().clone())
            y_offset.append(_input[i][0][1].detach().clone())
    
        for j in range(_input.shape[0]):
            for i in range(_input.shape[1]):
                _input[j][i][0] = _input[j][i][0] - x_offset[j]
                _input[j][i][1] = _input[j][i][1] - y_offset[j]

        _input = _input.to(device)

        # Set to eval mode
        encoder.eval()
        decoder.eval()

        # Encoder
        batch_size = _input.shape[0]
        input_length = _input.shape[1]
        input_shape = _input.shape[2]

        # Initialize encoder hidden state
        encoder_hidden = (torch.zeros(batch_size, encoder.module.hidden_size).to(device), 
                          torch.zeros(batch_size, encoder.module.hidden_size).to(device))
       
        # Encode observed trajectory
        for ei in range(input_length):
            encoder_input = _input[:, ei, :]
            encoder_hidden = encoder(encoder_input, encoder_hidden)

        # Initialize decoder input with last coordinate in encoder
        decoder_input = encoder_input[:, :2]

        # Initialize decoder hidden state as encoder hidden state
        decoder_hidden = encoder_hidden

        decoder_outputs = torch.zeros(
            (batch_size, 30, 2)).to(device)

        # Decode hidden state in future trajectory
        for di in range(30):
            decoder_output, decoder_hidden = decoder(decoder_input,
                                                     decoder_hidden)
            decoder_outputs[:, di, :] = decoder_output

            # Use own predictions as inputs at next step
            decoder_input = decoder_output

        for i in range(30):
            for j in range(batch_size):
                decoder_outputs[j,i,0] = decoder_outputs[j,i,0] + x_offset[j]
                decoder_outputs[j,i,1] = decoder_outputs[j,i,1] + y_offset[j]
            
                if (idx[j][0][0][0] in forecasted_trajectories):
                    forecasted_trajectories[idx[j][0][0][0]].append(decoder_outputs[j,i,:].tolist())
                else:
                    forecasted_trajectories[idx[j][0][0][0]] = [decoder_outputs[j,i,:].tolist()]
                
    return(forecasted_trajectories)

In [8]:
output = infer_absolute(test_loader, encoder, decoder)

In [9]:
import pandas as pd
df = pd.DataFrame.from_dict(output, orient='index')
df

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
8756,"[724.0888061523438, 1068.1959228515625]","[724.0033569335938, 1068.7257080078125]","[724.1530151367188, 1068.6138916015625]","[724.2235107421875, 1068.80517578125]","[724.3062744140625, 1069.01318359375]","[724.423828125, 1069.245849609375]","[724.525146484375, 1069.4951171875]","[724.6327514648438, 1069.750732421875]","[724.7332153320312, 1070.0103759765625]","[724.830322265625, 1070.2706298828125]",...,"[725.6737060546875, 1073.2012939453125]","[725.7422485351562, 1073.4949951171875]","[725.8108520507812, 1073.7958984375]","[725.8795166015625, 1074.1041259765625]","[725.9483642578125, 1074.419921875]","[726.0175170898438, 1074.7430419921875]","[726.0870361328125, 1075.0736083984375]","[726.1567993164062, 1075.4111328125]","[726.2267456054688, 1075.75537109375]","[726.2967529296875, 1076.105712890625]"
11804,"[267.89501953125, 654.7109985351562]","[267.67840576171875, 654.027587890625]","[267.97564697265625, 653.3213500976562]","[267.8216247558594, 652.5756225585938]","[267.9857177734375, 651.898193359375]","[267.9341125488281, 651.1454467773438]","[268.0383605957031, 650.4575805664062]","[268.03668212890625, 649.720947265625]","[268.10247802734375, 649.0230712890625]","[268.11895751953125, 648.302490234375]",...,"[268.40765380859375, 640.5118408203125]","[268.4488220214844, 639.7854614257812]","[268.4950866699219, 639.0528564453125]","[268.54644775390625, 638.3138427734375]","[268.602783203125, 637.568115234375]","[268.663818359375, 636.8156127929688]","[268.7292175292969, 636.056396484375]","[268.79852294921875, 635.2905883789062]","[268.8712463378906, 634.518310546875]","[268.94671630859375, 633.7396850585938]"
14463,"[1735.7835693359375, 455.59405517578125]","[1735.150146484375, 456.0295715332031]","[1734.731689453125, 456.0682678222656]","[1734.081298828125, 456.0626525878906]","[1733.3828125, 456.0826416015625]","[1732.6666259765625, 456.09136962890625]","[1731.9481201171875, 456.0870361328125]","[1731.2222900390625, 456.06640625]","[1730.487548828125, 456.02996826171875]","[1729.7435302734375, 455.9793701171875]",...,"[1720.4716796875, 455.2750549316406]","[1719.69091796875, 455.3455810546875]","[1718.9219970703125, 455.44049072265625]","[1718.1614990234375, 455.55419921875]","[1717.4088134765625, 455.68359375]","[1716.664794921875, 455.8272705078125]","[1715.931640625, 455.9847412109375]","[1715.2119140625, 456.15594482421875]","[1714.508544921875, 456.3407897949219]","[1713.8248291015625, 456.5389099121094]"
5166,"[592.3160400390625, 1048.3477783203125]","[592.2496948242188, 1047.864501953125]","[592.410400390625, 1047.433349609375]","[592.3189086914062, 1046.9652099609375]","[592.3997802734375, 1046.486572265625]","[592.3566284179688, 1046.017333984375]","[592.4102783203125, 1045.5596923828125]","[592.3914184570312, 1045.100830078125]","[592.4267578125, 1044.6591796875]","[592.4191284179688, 1044.21826171875]",...,"[592.4694213867188, 1040.0494384765625]","[592.4821166992188, 1039.7060546875]","[592.4989624023438, 1039.3626708984375]","[592.52001953125, 1039.017822265625]","[592.5458984375, 1038.670166015625]","[592.576904296875, 1038.3184814453125]","[592.613525390625, 1037.96142578125]","[592.6561889648438, 1037.5977783203125]","[592.705322265625, 1037.2264404296875]","[592.7612915039062, 1036.84619140625]"
16763,"[2183.27734375, 727.316162109375]","[2182.504638671875, 726.5989990234375]","[2181.649658203125, 725.9740600585938]","[2180.888427734375, 725.22900390625]","[2180.079833984375, 724.468994140625]","[2179.25732421875, 723.7000122070312]","[2178.424560546875, 722.93359375]","[2177.583740234375, 722.1723022460938]","[2176.73583984375, 721.4164428710938]","[2175.8818359375, 720.664794921875]",...,"[2166.43310546875, 712.2822875976562]","[2165.599853515625, 711.5074462890625]","[2164.775634765625, 710.7362670898438]","[2163.9609375, 709.9708251953125]","[2163.15673828125, 709.2133178710938]","[2162.363037109375, 708.465576171875]","[2161.58056640625,

In [10]:
for i in range(30):
    df[['v{}'.format((i*2)+1), 'v{}'.format((i*2)+2)]] = pd.DataFrame(df.get(i).tolist(), index=df.index)

In [11]:
import numpy as np
dropped_cols = list(np.arange(30))
df2 = df.drop(dropped_cols, axis=1)
df2.index.name = 'ID'
df2

,v1,v2,v3,v4,v5,v6,v7,v8,v9,v10,...,v51,v52,v53,v54,v55,v56,v57,v58,v59,v60
ID,,,,,,,,,,,,,,,,,,,,,
8756,724.088806,1068.195923,724.003357,1068.725708,724.153015,1068.613892,724.223511,1068.805176,724.306274,1069.013184,...,726.017517,1074.743042,726.087036,1075.073608,726.156799,1075.411133,726.226746,1075.755371,726.296753,1076.105713
11804,267.895020,654.710999,267.678406,654.027588,267.975647,653.321350,267.821625,652.575623,267.985718,651.898193,...,268.663818,636.815613,268.729218,636.056396,268.798523,635.290588,268.871246,634.518311,268.946716,633.739685
14463,1735.783569,455.594055,1735.150146,456.029572,1734.731689,456.068268,1734.081299,456.062653,1733.382812,456.082642,...,1716.664795,455.827271,1715.931641,455.984741,1715.211914,456.155945,1714.508545,456.340790,1713.824829,456.538910
5166,592.316040,1048.347778,592.249695,1047.864502,592.410400,1047.433350,592.318909,1046.965210,592.399780,1046.486572,...,592.576904,1038.318481,592.613525,1037.961426,592.656189,1037.597778,592.705322,1037.226440,592.761292,1036.846191
16763,2183.277344,727.316162,2182.504639,726.598999,2181.649658,725.974060,2180.888428,725.229004,2180.079834,724.468994,...,2162.363037,708.465576,2161.580566,707.729126,2160.809326,707.005005,2160.049316,706.293945,2159.300293,705.595886
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34491,2021.043335,601.891113,2019.940552,601.141296,2018.835327,600.351196,2017.865479,599.481689,2016.860107,598.597046,...,1996.411499,580.642151,1995.538086,579.878296,1994.675903,579.127075,1993.824829,578.387390,1992.984253,577.658020
31377,2184.509277,728.566101,2183.798340,727.946777,2182.977783,727.406677,2182.264160,726.739990,2181.492920,726.054871,...,2164.724121,711.234070,2163.975342,710.515015,2163.237305,709.802063,2162.509521,709.096680,2161.792725,708.399963
26011,2103.612549,661.581909,2102.357910,660.594971,2101.031982,659.510071,2099.821533,658.428406,2098.605713,657.352539,...,2073.830811,635.570679,2072.770752,634.662720,2071.723389,633.765320,2070.688721,632.878479,2069.666260,632.002808


In [12]:
df2.to_csv("outputs20epb6.csv", index=True)